In [1]:
import seaborn as sns
import numpy as np
import logging
from itertools import product
from copy import deepcopy, copy
from collections.abc import Sequence

In [2]:
from cosmicfishpie.fishermatrix import cosmicfish
from cosmicfishpie.LSSsurvey import photo_obs as pobs
from cosmicfishpie.LSSsurvey import photo_cov as pcov
from cosmicfishpie.likelihood import PhotometricLikelihood
from cosmicfishpie.utilities.utils import printing as upr
from nautilus import Prior
from nautilus import Sampler
import re
import time

In [3]:
snscolors = sns.color_palette("colorblind")
def is_indexable_iterable(var):
    return isinstance(var, (list, np.ndarray, Sequence)) and not isinstance(var, (str, bytes))


In [4]:
logger = logging.getLogger("cosmicfishpie.cosmology.nuisance")
logger.setLevel(logging.INFO)

In [5]:
upr.debug = False

In [6]:
upr.debug_print("test")

In [7]:
outroot = "cosmicjellyfish_Euclid-ISTF-Pess-GCphoto_symb_GW"

In [8]:
sampler_settings = {
    "n_live": 1000,
    "n_networks": 8,
    "n_batch": 256,
    "pool": 8,
}

In [9]:
fiducial = {'h': 0.6737}

fiducial = {
    "omch2": (0.3145-0.0491)*fiducial['h']**2,
    "ombh2": 0.0491*fiducial['h']**2,
    "h": 0.6737,
    "ns": 0.96605,
    "10^9As": 2.42,
}
freeparams = {kk:0.01 for kk in fiducial.keys()}



observables = ['GCph']

In [10]:
options = {
    "accuracy": 1,
    "feedback": 1,
    "code": "symbolic",
    "outroot": outroot,
    "survey_name": "Euclid",
    "survey_name_photo": "Euclid-Photometric-ISTF-GW",
    "survey_name_spectro": False,
    "specs_dir": "../cosmicfishpie/configs/other_survey_specifications/",
    "cosmo_model": "LCDM",
}
cosmoFM_fid = cosmicfish.FisherMatrix(
    fiducialpars=fiducial,
    options=options,
    observables=observables,
    freepars=freeparams,
    cosmoModel=options["cosmo_model"],
    surveyName=options["survey_name"],
)

****************************************************************
   _____               _     _____     __  
  / ___/__  ___ __ _  (_)___/ __(_)__ / /  
 / /__/ _ \(_-</  ' \/ / __/ _// (_-</ _ \ 
 \___/\___/___/_/_/_/_/\__/_/ /_/___/_//_/ 

****************************************************************
 This is the new Python version of the CosmicFish code.
****************************************************************

  -> Survey loaded:  Euclid-Photometric-ISTF-GW

  -> Computing cosmology at the fiducial point

  ---> Cosmological functions obtained in:   0.11 s


In [11]:
cosmoFM_fid.freeparams

{'omch2': 0.01,
 'ombh2': 0.01,
 'h': 0.01,
 'ns': 0.01,
 '10^9As': 0.01,
 'b0': 0.06}

In [12]:
cosmoFM_fid.fiducialcosmopars

{'omch2': 0.12045754652599999,
 'ombh2': 0.022285099978999995,
 'h': 0.6737,
 'ns': 0.96605,
 '10^9As': 2.42}

In [13]:
# #photo_fid = pobs.ComputeCls(cosmopars=cosmoFM_fid.fiducialcosmopars,
#                             photopars=cosmoFM_fid.photopars,
#                             IApars=cosmoFM_fid.IApars,
#                             biaspars=cosmoFM_fid.photobiaspars)

# #photo_fid.compute_all()

# #photo_cov_fid = pcov.PhotoCov(cosmopars=cosmoFM_fid.fiducialcosmopars,
#     #photopars=cosmoFM_fid.photopars,
#     #IApars=cosmoFM_fid.IApars,
#     #biaspars=cosmoFM_fid.photobiaspars,
#     #fiducial_Cls=photo_fid)

In [14]:
cosmoFM_fid.allparams

{'omch2': 0.12045754652599999,
 'ombh2': 0.022285099978999995,
 'h': 0.6737,
 'ns': 0.96605,
 '10^9As': 2.42,
 'bias_model': 'sqrt',
 'b0': 1.0,
 'fout': 0.1,
 'co': 1,
 'cb': 1,
 'sigma_o': 0.05,
 'sigma_b': 0.05,
 'zo': 0.1,
 'zb': 0.0,
 'IA_model': 'eNLA',
 'AIA': 1.72,
 'betaIA': 2.17,
 'etaIA': -0.41}

In [15]:
cosmoFM_fid.observables

['GCph']

In [16]:
fishmat_photo = cosmoFM_fid.compute()


In class: FisherMatrix  ----> Computing photo Fisher matrix



Computing fiducial

Fiducial generated in 0.46 s

Noise added to fiducial

Noisy Cls generated in 0.00 s

Computed covariance matrix

Covmat of Cls generated in 0.14 s

Total calculation in 0.60 s
>> computing derivs >>

  +++ Computing derivative on omch2

In class: derivatives  Derivative on omch2 done! in :  1.09 s

  +++ Computing derivative on ombh2

In class: derivatives  Derivative on ombh2 done! in :  1.10 s

  +++ Computing derivative on h

In class: derivatives  Derivative on h done! in :  1.09 s

  +++ Computing derivative on ns

In class: derivatives  Derivative on ns done! in :  1.02 s

  +++ Computing derivative on 10^9As

In class: derivatives  Derivative on 10^9As done! in :  1.02 s

  +++ Computing derivative on b0

In class: derivatives  Derivative on b0 done! in :  0.84 s

In class: PhotoCov  -->> Derivatives computed in   6.16 s

In class: FisherMatrix  Computing Fisher matrix

  Finished calculation of Fisher Matrix for ['GCph'] in:   0.01 s

  Fisher matrix calcu

In [17]:
photo_like = PhotometricLikelihood(cosmo_data=cosmoFM_fid, cosmo_theory=cosmoFM_fid)

In [18]:
ellmax_GC = cosmoFM_fid.specs["lmax_GCph"]
print(ellmax_GC)

1000


In [19]:
samp1dic = cosmoFM_fid.allparams.copy()

In [20]:
print("Sample likelihood", photo_like.loglike(param_dict=samp1dic))

samp1dic['b0'] = 1.1

print("Sample likelihood", photo_like.loglike(param_dict=samp1dic))

samp1dic['h'] = 0.7

print("Sample likelihood", photo_like.loglike(param_dict=samp1dic))

Sample likelihood -2.1673313981962915e-11
Sample likelihood -20442.036450933974
Sample likelihood -22680.93616163847


In [21]:
fiducial

{'omch2': 0.12045754652599999,
 'ombh2': 0.022285099978999995,
 'h': 0.6737,
 'ns': 0.96605,
 '10^9As': 2.42}

In [22]:
prior_dict ={
 'omch2': [(0.24-0.05)*0.67**2, (0.4-0.05)*0.67**2],
 'ombh2': [0.04*0.67**2, 0.06*0.67**2],
 'h': [0.61, 0.75],
 'ns': [0.92, 1.00],
 '10^9As': [2., 3.],
 'b0': [0.5, 1.5],
 }

In [23]:
prior_with_nuis = photo_like.create_nautilus_prior(prior_dict=prior_dict)

In [24]:
tini = time.time()
print("Starting sampler at", time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(tini)))

Starting sampler at 2025-09-30 12:26:18


In [29]:
sampler_settings.update({'pass_dict': False, 
                    'resume': True, 
                    'filepath': options["outroot"]+".hdf5", 
                    })
sampler_settings

{'n_live': 1000,
 'n_networks': 8,
 'n_batch': 256,
 'pool': 8,
 'pass_dict': False,
 'resume': True,
 'filepath': 'cosmicjellyfish_Euclid-ISTF-Pess-GCphoto_symb_GW.hdf5'}

In [30]:
sampler = photo_like.run_nautilus(prior=prior_with_nuis, sampler_kwargs=sampler_settings,
                                  run_kwargs={'verbose': True, 'discard_exploration': True})

Starting the nautilus sampler...
Please report issues at github.com/johannesulf/nautilus.
Status    | Bounds | Ellipses | Networks | Calls    | f_live | N_eff | log Z    
Finished  | 28     | 1        | 8        | 50944    | N/A    | 10129 | -17.60   


In [33]:
log_z_all = sampler.log_z
print('Evidence:', log_z_all)
points_all, log_w_all, log_l_all = sampler.posterior()

Evidence: -17.602029515296437


In [34]:
tfin = time.time()
elapsed = tfin - tini
hours = int(elapsed // 3600)
minutes = int((elapsed % 3600) // 60)
seconds = int(elapsed % 60)
print("Sampler finished at", time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(tfin)))
print(f"Total time elapsed: {hours:02d}:{minutes:02d}:{seconds:02d}")

Sampler finished at 2025-09-30 17:04:17
Total time elapsed: 04:37:59


In [35]:
sample_wghlkl = (np.vstack((points_all.T, np.exp(log_w_all), log_l_all)).T)

In [36]:
outfile_chain = options["outroot"]+".txt"
print(f"Saving chain to text file {outfile_chain}")

Saving chain to text file cosmicjellyfish_Euclid-ISTF-Pess-GCphoto_symb_GW.txt


In [ ]:
headerlist = list(prior_with_nuis.keys)+['loglike', 'weights'] 
header = " ".join(headerlist)
print("Saving header: ", header)

Saving header:  loglike weights omch2 ombh2 h ns 10^9As b0


In [38]:
np.savetxt(outfile_chain, sample_wghlkl, header=header)